In [96]:
import pandas as pd
import os
import hashlib
import psycopg2
from psycopg2.extras import execute_values
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

INPUT_DIR = '../Data Layer/raw'
OUTPUT_DIR = '../Data Layer/silver'


DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'brazilian-e-commerce',
    'user': 'admin',
    'password': 'admin123'
}

df_items = pd.read_csv(os.path.join(INPUT_DIR, "order_items.csv"))
df_orders = pd.read_csv(os.path.join(INPUT_DIR, "orders.csv"))
df_products = pd.read_csv(os.path.join(INPUT_DIR, "products.csv"))

In [97]:
nulls_before = df_products['product_category_name'].isna().sum()
df_products['product_category_name'] = df_products['product_category_name'].fillna('outros')
print(f"   Categorias nulas tratadas: {nulls_before}")

items_initial = len(df_items)

df_items = df_items[df_items["order_id"].isin(df_orders["order_id"])]

df_items = df_items[df_items["product_id"].isin(df_products["product_id"])]

print(f"   Registros órfãos removidos: {items_initial - len(df_items)}")

   Categorias nulas tratadas: 610
   Registros órfãos removidos: 0


In [98]:
df_silver = df_items.merge(df_orders, on='order_id', how='inner')
df_silver = df_silver.merge(df_products, on='product_id', how='inner')

cols_data = ['order_purchase_timestamp', 
             'order_approved_at', 
             'order_delivered_customer_date']

for col in cols_data:
    
    df_silver[col] = pd.to_datetime(df_silver[col], errors='coerce')

df_silver['product_category_name'] = df_silver['product_category_name'].str.replace('_', ' ').str.title()


def generate_sk(row):
    
    val = f"{row['order_id']}-{row['order_item_id']}"
    
    return hashlib.md5(val.encode()).hexdigest()

df_silver['sk_order_item'] = df_silver.apply(generate_sk, axis=1)


df_silver['total_item_value'] = df_silver['price'] + df_silver['freight_value']

df_silver['days_to_deliver'] = (df_silver['order_delivered_customer_date'] - df_silver['order_purchase_timestamp']).dt.days

def classificar_preco(preco):
    
    if preco < 50: return 'Budget'
    elif preco < 150: return 'Standard'
    elif preco < 500: return 'Premium'
    else: return 'Luxury'

df_silver['price_segment'] = df_silver['price'].apply(classificar_preco)

print(f"   Shape Final Silver: {df_silver.shape}")

   Shape Final Silver: (112650, 26)


In [ ]:
def executar_script_sql(arquivo_sql, cursor):
    
    try:
        with open(arquivo_sql, 'r', encoding='utf-8') as f:
            sql_script = f.read()
            cursor.execute(sql_script)
            print(f"Script executado: {arquivo_sql}")
    except Exception as e:
        print(f"Erro ao executar SQL: {e}")
        raise e

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

executar_script_sql('../Data Layer/silver/ddl.sql', cur)
conn.commit()
print("   Tabela silver.sales_order_items recriada com sucesso.")

cols_order = [
    'sk_order_item', 
    'order_id', 
    'order_item_id', 
    'product_id', 
    'seller_id',
    'price', 
    'freight_value', 
    'total_item_value', 
    'product_category_name', 
    'price_segment',
    'order_status', 
    'order_purchase_timestamp', 
    'days_to_deliver'
]

df_final = df_silver[cols_order] 

data_tuples = [tuple(x) for x in df_final.to_numpy()]

data_tuples = [tuple(None if pd.isna(x) else x for x in row) for row in data_tuples]


insert_query = """
    INSERT INTO silver.sales_order_items (
        sk_order_item, 
        order_id, 
        order_item_id, 
        product_id, seller_id,
        price, 
        freight_value, 
        total_item_value, 
        product_category_name, 
        price_segment,
        order_status, 
        order_purchase_timestamp, 
        days_to_deliver
    ) VALUES %s
"""

try:
    execute_values(cur, insert_query, data_tuples)
    conn.commit()
    print(f"{len(df_final):,} registros inseridos com sucesso!")
except Exception as e:
    conn.rollback()
    print(f"Erro na inserção: {e}")

cur.close()
conn.close()
print("\nETL CONCLUÍDA")

   ✅ Script executado: ../Data Layer/silver/ddl.sql
   Tabela silver.sales_order_items recriada com sucesso.
   🚀 112,650 registros inseridos com sucesso!

ETL CONCLUÍDA
